In [1]:
import requests # librería para manejar las solicitudes
import pandas as pd # librería para análisis de datos
import numpy as np # librería para manejar datos vectorizados
import random # librería para generar números aleatorios


#!pip install geopy
from geopy.geocoders import Nominatim # módulo para convertir una dirección en valores de latitud y longitud 

# librerías para mostrar imágenes 
from IPython.display import Image 
from IPython.core.display import HTML 
    
# librería para convertir un archivo json en un dataframe pandas
from pandas.io.json import json_normalize


#! pip install folium==0.5.0
import folium # librería para graficar 

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
mexico_coordenadas = pd.read_excel(r'C:\Users\luisp\Documents\Conjunto_de_datos_iter_00CSV20.xlsx')
column_names=['Número','Entidad federativa','Capital','Latitud','Longitud']
mexico_coordenadas = pd.DataFrame(mexico_coordenadas)
mexico_coordenadas.columns=column_names
mexico_coordenadas=mexico_coordenadas.drop(0,axis=0)
mexico_coordenadas=mexico_coordenadas.drop('Número',axis=1)
mexico_coordenadas

,Entidad federativa,Capital,Latitud,Longitud
1,Ciudad de México,Ciudad de México,19.42847,-99.12766
2,Aguascalientes,Aguascalientes,21.88234,-102.28259
3,Baja California,Mexicali,32.62781,-115.45446
4,Baja California Sur,La Paz,24.14437,-110.3005
5,Campeche,Campeche,19.84386,-90.52554
6,Chiapas,Tuxtla Gutiérrez,16.75973,-93.11308
7,Chihuahua,Chihuahua,28.63528,-106.08889
8,Coahuila de Zaragoza,Saltillo,25.42321,-101.0053
9,Colima,Colima,19.24997,-103.72714
10,Durango,Victoria de Durango,24.02032,-104.65756


In [3]:
# Obtner latitud y longitud de Colombia
address = 'Mexico'

geolocator = Nominatim(user_agent="mexico_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Las coordenadas geograficas de México son {}, {}.'.format(latitude, longitude))

Las coordenadas geograficas de México son 23.6585116, -102.0077097.


In [4]:
map_mexico = folium.Map(location=[latitude, longitude], zoom_start=5)

# añadir marcadores al mapa
for lat, lng, estado, capital in zip(mexico_coordenadas['Latitud'], mexico_coordenadas['Longitud'], mexico_coordenadas['Entidad federativa'], mexico_coordenadas['Capital']):
    label = '{}, {}'.format(capital, estado)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mexico)  
    
map_mexico

In [5]:
#Claves de acceso
CLIENT_ID = 'NS0FEXQSVPYHOWWS1YNVPJRV00ZXYTQAJHDBJME3CLK11JJC' # su ID de Foursquare
CLIENT_SECRET = 'BA5HGWEBAZWIZYC4LCEZPFZXNE41GKYJTOKF3UDEMIZCLQY0' # su Secreto de Cliente de Foursquare
VERSION = '20180604'
LIMIT=50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NS0FEXQSVPYHOWWS1YNVPJRV00ZXYTQAJHDBJME3CLK11JJC
CLIENT_SECRET:BA5HGWEBAZWIZYC4LCEZPFZXNE41GKYJTOKF3UDEMIZCLQY0


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):

    print("Estados consultados:")
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = "https://api.foursquare.com/v3/places/search?ll={},{}&radius={}&limit=50"

        headers = {
        "Accept": "application/json",
        "Authorization": "fsq3NP4mmQ7pPaGoquKKTT5AKga9Oc6mj10jP33RqlYAasA="
                }
        url = url.format(lat, lng,radius,LIMIT)
        data =  requests.request("GET",url, headers=headers).json()
        # do stuff here to process venues_list
        for record in data['results']:
            venues_list.append({
                'Entidad Federativa': name,
                'Latitude': lat,
                'Longitude': lng,
                'Lugar': record['name'],
                'Lugar Latitud': record['geocodes']['main']['latitude'],
                'Lugar Longitud': record['geocodes']['main']['longitude'],
                'Lugar Categoría': record['categories'][0]['name'],
            })

    # do stuff here to create your dataframe
    nearby_venues = pd.DataFrame(venues_list)

    return(nearby_venues)

In [9]:
lugares_cercanos = getNearbyVenues(names=mexico_coordenadas['Entidad federativa'],
                                   latitudes=mexico_coordenadas['Latitud'],
                                   longitudes=mexico_coordenadas['Longitud']
                                  )

Estados consultados:
Ciudad de México
Aguascalientes
Baja California
Baja California Sur
Campeche
Chiapas
Chihuahua
Coahuila de Zaragoza
Colima
Durango
Guanajuato
Guerrero
Hidalgo
Jalisco
México
Michoacán de Ocampo
Morelos
Nayarit
Nuevo León
Oaxaca
Puebla
Querétaro Arteaga
Quintana Roo
San Luis Potosí
Sinaloa
Sonora
Tabasco
Tamaulipas
Tlaxcala
Veracruz
Yucatán
Zacatecas


In [10]:
# Tamaño del dataframe.
print('El dataframe tiene {} filas'.format(lugares_cercanos.shape[0]), 'y {} columnas'.format(lugares_cercanos.shape[1]))
#Verificar si existen na
print('\n', 'Conteo de NaN')
print('\n',lugares_cercanos.isnull().sum(axis=0))

El dataframe tiene 1561 filas y 7 columnas

 Conteo de NaN

 Entidad Federativa    0
Latitude              0
Longitude             0
Lugar                 0
Lugar Latitud         0
Lugar Longitud        0
Lugar Categoría       0
dtype: int64


In [11]:
lugares_cercanos.head(60)

,Entidad Federativa,Latitude,Longitude,Lugar,Lugar Latitud,Lugar Longitud,Lugar Categoría
0,Ciudad de México,19.42847,-99.12766,Al Andalus,19.427718,-99.129169,Lebanese Restaurant
1,Ciudad de México,19.42847,-99.12766,El Antiguo Edhen,19.430239,-99.129345,Falafel Restaurant
2,Ciudad de México,19.42847,-99.12766,Fantasías Miguel,19.429902,-99.131416,Retail
3,Ciudad de México,19.42847,-99.12766,Sunday Sunday,19.430450,-99.131344,Music Venue
4,Ciudad de México,19.42847,-99.12766,Museo Nacional de las Culturas,19.433236,-99.130408,History Museum
5,Ciudad de México,19.42847,-99.12766,Murales de Diego Rivera en la Secretaría de Ed...,19.437165,-99.131413,Art Museum
6,Ciudad de México,19.42847,-99.12766,El Palacio de Hierro,19.430864,-99.133950,Boutique
7,Ciudad de México,19.42847,-99.12766,El Palacio de Hierro,19.431112,-99.134147,Boutique
8,Ciudad de México,19.42847,-99.12766,Templo Mayor (Museo del Templo Mayor),19.434582,-99.131938,Landmarks and Outdoors
9,Ciudad de México,19.42847,-99.12766,Ex Teresa Arte Actual,19.434167,-99.130666,Art Museum


In [12]:
# Numero de lugares consultados en cada capital de los estados de México
lugares_cercanos.groupby('Entidad Federativa').count()

,Latitude,Longitude,Lugar,Lugar Latitud,Lugar Longitud,Lugar Categoría
Entidad Federativa,,,,,,
Aguascalientes,50,50,50,50,50,50
Baja California,50,50,50,50,50,50
Baja California Sur,50,50,50,50,50,50
Campeche,50,50,50,50,50,50
Chiapas,50,50,50,50,50,50
Chihuahua,50,50,50,50,50,50
Ciudad de México,50,50,50,50,50,50
Coahuila de Zaragoza,50,50,50,50,50,50
Colima,50,50,50,50,50,50


In [13]:
print('Hay {} categorias.'.format(len(lugares_cercanos['Lugar Categoría'].unique())))

Hay 183 categorias.


In [14]:
#Análisis de cada municipio

mexico_lugares_onehot = pd.get_dummies(lugares_cercanos[['Lugar Categoría']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
mexico_lugares_onehot['Entidad Federativa'] = lugares_cercanos['Entidad Federativa'] 

# mover la columna de barrio a la primer columna
fixed_columns = [mexico_lugares_onehot.columns[-1]] + list(mexico_lugares_onehot.columns[:-1])
mexico_lugares_onehot = mexico_lugares_onehot[fixed_columns]

mexico_lugares_onehot.head()



,Entidad Federativa,American Restaurant,Antique Store,Argentinian Restaurant,Art Gallery,Art Museum,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Automotive Repair Shop,...,Theme Restaurant,Toy / Game Store,Vegan and Vegetarian Restaurant,Veterinarian,Warehouse / Wholesale Store,Water Park,Wine Bar,Wings Joint,Yucatecan Restaurant,Zoo
0,Ciudad de México,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ciudad de México,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ciudad de México,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ciudad de México,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ciudad de México,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#Tamño del dataframe
print('El dataframe tiene {} filas'.format(mexico_lugares_onehot.shape[0]), 'y {} columnas'.format(mexico_lugares_onehot.shape[1]))

El dataframe tiene 1561 filas y 184 columnas


In [16]:
#Agrupemos las filas por barrios tomando la média de la frecuancia de la ocurrencia de cada categoría
mexico_lugares_grouped = mexico_lugares_onehot.groupby('Entidad Federativa').mean().reset_index()
mexico_lugares_grouped

,Entidad Federativa,American Restaurant,Antique Store,Argentinian Restaurant,Art Gallery,Art Museum,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Automotive Repair Shop,...,Theme Restaurant,Toy / Game Store,Vegan and Vegetarian Restaurant,Veterinarian,Warehouse / Wholesale Store,Water Park,Wine Bar,Wings Joint,Yucatecan Restaurant,Zoo
0,Aguascalientes,0.00,0.00,0.00,0.00,0.04,0.00,0.04,0.00,0.00,...,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00
1,Baja California,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Baja California Sur,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Campeche,0.02,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.00,...,0.02,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Chiapas,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Chihuahua,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,Ciudad de México,0.00,0.00,0.00,0.00,0.06,0.02,0.02,0.00,0.00,...,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Coahuila de Zaragoza,0.00,0.00,0.00,0.04,0.04,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00
8,Colima,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00
9,Durango,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [17]:
#Lugares más populares por cada entidad
num_top_venues = 5

for hood in mexico_lugares_grouped['Entidad Federativa']:
    print("----"+hood+"----")
    temp = mexico_lugares_grouped[mexico_lugares_grouped['Entidad Federativa'] == hood].T.reset_index()
    temp.columns = ['Clasificación del Lugar','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aguascalientes----
  Clasificación del Lugar  freq
0      Mexican Restaurant  0.18
1         Taco Restaurant  0.08
2                 Stadium  0.06
3                     Bar  0.06
4        Ice Cream Parlor  0.06


----Baja California----
     Clasificación del Lugar  freq
0                Coffee Shop  0.08
1  Health and Beauty Service  0.06
2         Mexican Restaurant  0.06
3         Seafood Restaurant  0.06
4                        Bar  0.04


----Baja California Sur----
  Clasificación del Lugar  freq
0      Seafood Restaurant  0.08
1      Mexican Restaurant  0.08
2              Restaurant  0.08
3             Coffee Shop  0.06
4        Ice Cream Parlor  0.04


----Campeche----
  Clasificación del Lugar  freq
0      Mexican Restaurant  0.12
1                    Park  0.08
2         Taco Restaurant  0.06
3             Coffee Shop  0.06
4          Hardware Store  0.04


----Chiapas----
  Clasificación del Lugar  freq
0         Taco Restaurant  0.22
1              Restaurant  0.08
2 

In [18]:
#Función para obtener los lugares más comúnes
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
# Primeros 10 sitios de cada entidad
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Entidad Federativa']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Lugar más común'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Lugar más común'.format(ind+1))

# crear un nuevo dataframe
mexico_lugares_comunes = pd.DataFrame(columns=columns)
mexico_lugares_comunes ['Entidad Federativa'] = mexico_lugares_grouped['Entidad Federativa']

for ind in np.arange(mexico_lugares_grouped.shape[0]):
    mexico_lugares_comunes.iloc[ind, 1:] = return_most_common_venues(mexico_lugares_grouped.iloc[ind, :], num_top_venues)

mexico_lugares_comunes.head(10)

,Entidad Federativa,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4th Lugar más común,5th Lugar más común,6th Lugar más común,7th Lugar más común,8th Lugar más común,9th Lugar más común,10th Lugar más común
0,Aguascalientes,Mexican Restaurant,Taco Restaurant,Stadium,Bar,Ice Cream Parlor,Art Museum,Café,Arts and Entertainment,Restaurant,Plaza
1,Baja California,Coffee Shop,Health and Beauty Service,Mexican Restaurant,Seafood Restaurant,Bar,Restaurant,Salad Restaurant,Café,Fast Food Restaurant,Sushi Restaurant
2,Baja California Sur,Seafood Restaurant,Mexican Restaurant,Restaurant,Coffee Shop,Ice Cream Parlor,Asian Restaurant,Taco Restaurant,Bakery,Bar,Pizzeria
3,Campeche,Mexican Restaurant,Park,Taco Restaurant,Coffee Shop,Hardware Store,Pizzeria,Movie Theater,Dining and Drinking,Retail,Sandwich Restaurant
4,Chiapas,Taco Restaurant,Restaurant,Pizzeria,Seafood Restaurant,BBQ Joint,Food Truck,Park,Mexican Restaurant,Deli,Sushi Restaurant
5,Chihuahua,Coffee Shop,Mexican Restaurant,Taco Restaurant,Ice Cream Parlor,Restaurant,Seafood Restaurant,Café,Snack Place,Italian Restaurant,Pool Hall
6,Ciudad de México,Mexican Restaurant,Art Museum,Bookstore,Concert Hall,Historic and Protected Site,Bakery,Café,Taco Restaurant,Spanish Restaurant,Restaurant
7,Coahuila de Zaragoza,Plaza,Mexican Restaurant,Burger Joint,Bakery,Art Gallery,Art Museum,Taco Restaurant,History Museum,BBQ Joint,Bank
8,Colima,Mexican Restaurant,Taco Restaurant,Park,BBQ Joint,Bakery,Dining and Drinking,Seafood Restaurant,Grocery Store / Supermarket,Ice Cream Parlor,Clothing Store
9,Durango,Mexican Restaurant,Café,Plaza,Fast Food Restaurant,Bar,Coffee Shop,Taco Restaurant,Burger Joint,Landmarks and Outdoors,Middle Eastern Restaurant


In [20]:
#Ejecutemos k-means para agrupar los barrios en 5 agrupaciones.

# K-means para agrupar los municipios 
kclusters = 5

principales_grouped_clustering =mexico_lugares_grouped.drop('Entidad Federativa', 1)

# Ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(principales_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:10] 

C:\Users\luisp\AppData\Local\Temp\ipykernel_17100\3983025809.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  principales_grouped_clustering =mexico_lugares_grouped.drop('Entidad Federativa', 1)


array([0, 1, 1, 1, 4, 1, 0, 3, 4, 3])

In [21]:
# Los 10 sitios mas populares agrupados (K-means)
mexico_lugares_comunes.insert(0, 'Grupo', kmeans.labels_)

lugares_union = mexico_coordenadas

# Unir principales_grouped con principales_data 
lugares_union = lugares_union.join(mexico_lugares_comunes.set_index('Entidad Federativa'), on='Entidad federativa')

lugares_union

,Entidad federativa,Capital,Latitud,Longitud,Grupo,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4th Lugar más común,5th Lugar más común,6th Lugar más común,7th Lugar más común,8th Lugar más común,9th Lugar más común,10th Lugar más común
1,Ciudad de México,Ciudad de México,19.42847,-99.12766,0,Mexican Restaurant,Art Museum,Bookstore,Concert Hall,Historic and Protected Site,Bakery,Café,Taco Restaurant,Spanish Restaurant,Restaurant
2,Aguascalientes,Aguascalientes,21.88234,-102.28259,0,Mexican Restaurant,Taco Restaurant,Stadium,Bar,Ice Cream Parlor,Art Museum,Café,Arts and Entertainment,Restaurant,Plaza
3,Baja California,Mexicali,32.62781,-115.45446,1,Coffee Shop,Health and Beauty Service,Mexican Restaurant,Seafood Restaurant,Bar,Restaurant,Salad Restaurant,Café,Fast Food Restaurant,Sushi Restaurant
4,Baja California Sur,La Paz,24.14437,-110.3005,1,Seafood Restaurant,Mexican Restaurant,Restaurant,Coffee Shop,Ice Cream Parlor,Asian Restaurant,Taco Restaurant,Bakery,Bar,Pizzeria
5,Campeche,Campeche,19.84386,-90.52554,1,Mexican Restaurant,Park,Taco Restaurant,Coffee Shop,Hardware Store,Pizzeria,Movie Theater,Dining and Drinking,Retail,Sandwich Restaurant
6,Chiapas,Tuxtla Gutiérrez,16.75973,-93.11308,4,Taco Restaurant,Restaurant,Pizzeria,Seafood Restaurant,BBQ Joint,Food Truck,Park,Mexican Restaurant,Deli,Sushi Restaurant
7,Chihuahua,Chihuahua,28.63528,-106.08889,1,Coffee Shop,Mexican Restaurant,Taco Restaurant,Ice Cream Parlor,Restaurant,Seafood Restaurant,Café,Snack Place,Italian Restaurant,Pool Hall
8,Coahuila de Zaragoza,Saltillo,25.42321,-101.0053,3,Plaza,Mexican Restaurant,Burger Joint,Bakery,Art Gallery,Art Museum,Taco Restaurant,History Museum,BBQ Joint,Bank
9,Colima,Colima,19.24997,-103.72714,4,Mexican Restaurant,Taco Restaurant,Park,BBQ Joint,Bakery,Dining and Drinking,Seafood Restaurant,Grocery Store / Supermarket,Ice Cream Parlor,Clothing Store
10,Durango,Victoria de Durango,24.02032,-104.65756,3,Mexican Restaurant,Café,Plaza,Fast Food Restaurant,Bar,Coffee Shop,Taco Restaurant,Burger Joint,Landmarks and Outdoors,Middle Eastern Restaurant


In [22]:
# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(lugares_union ['Latitud'], lugares_union ['Longitud'], lugares_union ['Entidad federativa'],lugares_union ['Grupo']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [23]:
# Agrupacion 1
lugares_union.loc[lugares_union['Grupo'] == 0, lugares_union.columns[[0] + list(range(5, lugares_union.shape[1]))]]

,Entidad federativa,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4th Lugar más común,5th Lugar más común,6th Lugar más común,7th Lugar más común,8th Lugar más común,9th Lugar más común,10th Lugar más común
1,Ciudad de México,Mexican Restaurant,Art Museum,Bookstore,Concert Hall,Historic and Protected Site,Bakery,Café,Taco Restaurant,Spanish Restaurant,Restaurant
2,Aguascalientes,Mexican Restaurant,Taco Restaurant,Stadium,Bar,Ice Cream Parlor,Art Museum,Café,Arts and Entertainment,Restaurant,Plaza
12,Guerrero,Mexican Restaurant,Taco Restaurant,Restaurant,Pizzeria,Burger Joint,Café,Coffee Shop,Drugstore,Sushi Restaurant,Park
17,Morelos,Mexican Restaurant,Coffee Shop,Café,Taco Restaurant,Bar,Art Museum,Health and Beauty Service,French Restaurant,Grocery Store / Supermarket,Brewery
20,Oaxaca,Mexican Restaurant,Brewery,Coffee Shop,Art Gallery,Bar,Arts and Crafts Store,Arts and Entertainment,Restaurant,Cocktail Bar,Breakfast Spot
21,Puebla,Mexican Restaurant,Coffee Shop,Bar,Art Museum,Ice Cream Parlor,Candy Store,Pizzeria,Steakhouse,Bookstore,Pastry Shop
22,Querétaro Arteaga,Mexican Restaurant,Plaza,Taco Restaurant,Cocktail Bar,History Museum,Health and Beauty Service,Art Museum,Bakery,Brewery,Restaurant
23,Quintana Roo,Mexican Restaurant,Taco Restaurant,Seafood Restaurant,Restaurant,Burger Joint,Deli,Cupcake Shop,Drugstore,Education,Coffee Shop
28,Tamaulipas,Mexican Restaurant,Taco Restaurant,Pizzeria,Seafood Restaurant,Bistro,Health and Beauty Service,Movie Theater,Fast Food Restaurant,Clothing Store,Burger Joint
30,Veracruz,Mexican Restaurant,Café,Bar,Pizzeria,Bakery,Bookstore,Restaurant,Taco Restaurant,Seafood Restaurant,Tattoo Parlor


In [24]:
# Examinar Agrupacion 2
lugares_union.loc[lugares_union['Grupo'] == 1, lugares_union.columns[[0] + list(range(5, lugares_union.shape[1]))]]

,Entidad federativa,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4th Lugar más común,5th Lugar más común,6th Lugar más común,7th Lugar más común,8th Lugar más común,9th Lugar más común,10th Lugar más común
3,Baja California,Coffee Shop,Health and Beauty Service,Mexican Restaurant,Seafood Restaurant,Bar,Restaurant,Salad Restaurant,Café,Fast Food Restaurant,Sushi Restaurant
4,Baja California Sur,Seafood Restaurant,Mexican Restaurant,Restaurant,Coffee Shop,Ice Cream Parlor,Asian Restaurant,Taco Restaurant,Bakery,Bar,Pizzeria
5,Campeche,Mexican Restaurant,Park,Taco Restaurant,Coffee Shop,Hardware Store,Pizzeria,Movie Theater,Dining and Drinking,Retail,Sandwich Restaurant
7,Chihuahua,Coffee Shop,Mexican Restaurant,Taco Restaurant,Ice Cream Parlor,Restaurant,Seafood Restaurant,Café,Snack Place,Italian Restaurant,Pool Hall
13,Hidalgo,Taco Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Burger Joint,Café,Restaurant,Ice Cream Parlor,Buffet,Speakeasy
14,Jalisco,Seafood Restaurant,Dessert Shop,Taco Restaurant,Ice Cream Parlor,Mexican Restaurant,Coffee Shop,Restaurant,Spa,Italian Restaurant,Burger Joint
15,México,Mexican Restaurant,Taco Restaurant,Ice Cream Parlor,Café,Italian Restaurant,Seafood Restaurant,Restaurant,Coffee Shop,Plaza,Bakery
18,Nayarit,Taco Restaurant,Ice Cream Parlor,Restaurant,Café,Seafood Restaurant,Juice Bar,BBQ Joint,Bakery,Mexican Restaurant,Vegan and Vegetarian Restaurant
26,Sonora,Taco Restaurant,Coffee Shop,Seafood Restaurant,Fast Food Restaurant,Burger Joint,Ice Cream Parlor,Bakery,Park,Hot Dog Joint,Department Store
31,Yucatán,Café,Bar,Restaurant,History Museum,Mexican Restaurant,Bakery,Ice Cream Parlor,Historic and Protected Site,Museum,Park


In [25]:
# Examinar Agrupacion 3
lugares_union.loc[lugares_union['Grupo'] == 2, lugares_union.columns[[0] + list(range(5, lugares_union.shape[1]))]]

,Entidad federativa,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4th Lugar más común,5th Lugar más común,6th Lugar más común,7th Lugar más común,8th Lugar más común,9th Lugar más común,10th Lugar más común
25,Sinaloa,Mexican Restaurant,Landmarks and Outdoors,Restaurant,Burger Joint,Campground,Bakery,Seafood Restaurant,American Restaurant,Paella Restaurant,Nail Salon


In [26]:
# Examinar Agrupacion 4
lugares_union.loc[lugares_union['Grupo'] == 3, lugares_union.columns[[0] + list(range(5, lugares_union.shape[1]))]]

,Entidad federativa,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4th Lugar más común,5th Lugar más común,6th Lugar más común,7th Lugar más común,8th Lugar más común,9th Lugar más común,10th Lugar más común
8,Coahuila de Zaragoza,Plaza,Mexican Restaurant,Burger Joint,Bakery,Art Gallery,Art Museum,Taco Restaurant,History Museum,BBQ Joint,Bank
10,Durango,Mexican Restaurant,Café,Plaza,Fast Food Restaurant,Bar,Coffee Shop,Taco Restaurant,Burger Joint,Landmarks and Outdoors,Middle Eastern Restaurant
11,Guanajuato,Plaza,Café,Mexican Restaurant,Art Museum,Restaurant,French Restaurant,Speakeasy,Bubble Tea Shop,Building / Structure,Burrito Restaurant
16,Michoacán de Ocampo,Mexican Restaurant,Plaza,History Museum,Dining and Drinking,Landmarks and Outdoors,Bakery,Bar,Taco Restaurant,Coffee Shop,Gastropub
24,San Luis Potosí,Plaza,Mexican Restaurant,Bar,Café,Dessert Shop,Wings Joint,Candy Store,Dining and Drinking,Breakfast Spot,Monument
32,Zacatecas,Mexican Restaurant,Park,Restaurant,Museum,Plaza,Argentinian Restaurant,Ice Cream Parlor,Coffee Shop,Other Great Outdoors,Bar


In [27]:
# Examinar Agrupacion 5
lugares_union.loc[lugares_union['Grupo'] == 4, lugares_union.columns[[0] + list(range(5, lugares_union.shape[1]))]]

,Entidad federativa,1st Lugar más común,2nd Lugar más común,3rd Lugar más común,4th Lugar más común,5th Lugar más común,6th Lugar más común,7th Lugar más común,8th Lugar más común,9th Lugar más común,10th Lugar más común
6,Chiapas,Taco Restaurant,Restaurant,Pizzeria,Seafood Restaurant,BBQ Joint,Food Truck,Park,Mexican Restaurant,Deli,Sushi Restaurant
9,Colima,Mexican Restaurant,Taco Restaurant,Park,BBQ Joint,Bakery,Dining and Drinking,Seafood Restaurant,Grocery Store / Supermarket,Ice Cream Parlor,Clothing Store
19,Nuevo León,Mexican Restaurant,Taco Restaurant,Argentinian Restaurant,Plaza,Bakery,History Museum,Café,Seafood Restaurant,Dining and Drinking,Restaurant
27,Tabasco,Mexican Restaurant,Taco Restaurant,Ice Cream Parlor,Health and Beauty Service,Pizzeria,Argentinian Restaurant,Juice Bar,Seafood Restaurant,Drugstore,Bakery
29,Tlaxcala,Taco Restaurant,Mexican Restaurant,Steakhouse,BBQ Joint,Landmarks and Outdoors,Health and Beauty Service,Restaurant,Ice Cream Parlor,Coffee Shop,Seafood Restaurant
